# GPT Inference
- Using https://github.com/EleutherAI/gpt-neo open source implementation of GPT-3 2.7B model

## Setup

In [ ]:
%%capture
%tensorflow_version 2.x
!git clone https://github.com/EleutherAI/GPTNeo
%cd GPTNeo
!pip3 install -q -r requirements.txt
pretrained_model = None
dataset = None


# GCP Setup 

In [ ]:
from google.colab import auth
auth.authenticate_user()
!gcloud init

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'
core:
  account: novelspaceonly@gmail.com

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

Choose the account you would like to use to perform operations for 
this configuration:
 [1] novelspaceonly@gmail.com
 [2] Log in with a new account
Please enter your numeric choice:  1

You are logged in as: [novelspaceonly@gmail.com].

Pick cloud project to use: 
 [1

In [ ]:
path_to_cloud_bucket = 'gs://gpt-neo-chill/' #@param {type:"string"}

# Download pretrained model weights:

In [ ]:

pretrained_model = 'GPT3_2-7B' #@param ["GPT3_XL", "GPT3_2-7B"]
!wget -m -np -c -U "eye02" -w 2 -R "index.html*" "https://the-eye.eu/public/AI/gptneo-release/$pretrained_model/"
path_to_local_weights = f"/content/GPTNeo/the-eye.eu/public/AI/gptneo-release/{pretrained_model}"



--2021-03-28 20:15:57--  https://the-eye.eu/public/AI/gptneo-release/GPT3_2-7B/
Resolving the-eye.eu (the-eye.eu)... 162.213.130.242
Connecting to the-eye.eu (the-eye.eu)|162.213.130.242|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘the-eye.eu/public/AI/gptneo-release/GPT3_2-7B/index.html.tmp’

the-eye.eu/public/A     [ <=>                ]  14.42K  --.-KB/s    in 0.03s   

Last-modified header missing -- time-stamps turned off.
2021-03-28 20:15:57 (462 KB/s) - ‘the-eye.eu/public/AI/gptneo-release/GPT3_2-7B/index.html.tmp’ saved [14764]

Loading robots.txt; please ignore errors.
--2021-03-28 20:15:59--  https://the-eye.eu/robots.txt
Reusing existing connection to the-eye.eu:443.
HTTP request sent, awaiting response... 200 OK
Length: 4 [text/plain]
Saving to: ‘the-eye.eu/robots.txt’

the-eye.eu/robots.t 100%[===================>]       4  --.-KB/s    in 0s      

2021-03-28 20:15:59 (1.37 MB/s) - ‘the-eye.eu/robots.txt’ sav

# GCP Login

In [ ]:
!gcloud auth login 

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=He7CBcOUXIfULkDiwVyUxbd5976HzX&prompt=consent&access_type=offline&code_challenge=PpOBQR_Vp7lR27WdRf2M1TM_BrqOq7OJ2ZTmNvy0gH4&code_challenge_method=S256

Enter verification code: 4/1AY0e-g4eTWECjksCBUk7I7m-lpydd-pfG169Im2Qlgu_DRe87oBABzyjM6Q

You are now logged in as [novelspaceonly@gmail.com].
Your current project is [gpt-neo-chill].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


# Upload to Bucket

In [ ]:
!yum install gcc python3-devel python3-setuptools redhat-rpm-config
!sudo pip3 uninstall crcmod
!sudo pip3 install --no-cache-dir -U crcmod

/bin/bash: yum: command not found
Uninstalling crcmod-1.7:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/crcmod-1.7.dist-info/*
    /usr/local/lib/python3.7/dist-packages/crcmod/*
Proceed (y/n)? y
  Successfully uninstalled crcmod-1.7
     |████████████████████████████████| 92kB 6.7MB/s 
  Created wheel for crcmod: filename=crcmod-1.7-cp37-cp37m-linux_x86_64.whl size=36303 sha256=82989d32b11388c7533c376e7b13814f706d7b8ee0c1c5283bc4870763b52082
  Stored in directory: /tmp/pip-ephem-wheel-cache-mdo7abmr/wheels/50/24/4d/4580ca4a299f1ad6fd63443e6e584cb21e9a07988e4aa8daac
Successfully built crcmod


In [ ]:
# upload to your bucket
bucket_base = "gs://" + path_to_cloud_bucket.replace('gs://', '').split('/')[0]

In [ ]:
!gsutil -m cp -r $path_to_local_weights $bucket_base

Check to see if the upload to the bucket was successful

In [ ]:
!gsutil ls $bucket_base

In [ ]:
# @title Modify config for colab. 
  
import json
from pprint import pprint

path_to_model = "" #@param {type:"string"}
batch_size = 8 #@param {type:"integer"}
dset = "Sampling_Only"  #@param {type:"string"}
mesh_shape = "x:4,y:2" #@param {type:"string"}
train_steps = 1000 #@param {type:"integer"}
steps_per_checkpoint = 500 #@param {type:"integer"}
start_step = 400000 if pretrained_model == "GPT3_2-7B" else 362000

if path_to_model == "":
  path_to_model = f'{bucket_base.strip("/")}/{pretrained_model}'
print(f'MODEL PATH: {path_to_model}\n')

if dset == "" and dataset != "Sampling_Only":
  dset = dataset
elif dataset is None:
  dset = "pile"

def pad_to_multiple_of(n, mult):
  """
  pads n to a multiple of mult
  """
  extra = n % mult
  if extra > 0:
      n = n + mult - extra
  return n

with open(f'{path_to_local_weights}/config.json', 'r') as f:
  data = json.load(f)
  pprint(data)
  dset_val = [[dset, None, None, None]] if dset != "" else data["datasets"]
  mods = {
          "mesh_shape": mesh_shape,
          "layout": "intermediate_expanded:x,heads:x,memory_length:y,embd:y",
          "model_path": path_to_model,
          "datasets": dset_val,
          "train_steps": start_step + train_steps,
          "eval_steps": 0,
          "train_batch_size": batch_size,
          "predict_batch_size": batch_size
        }
  data.update(mods)
  print('\n--->\n')
  pprint(data)
  with open(f'configs/{pretrained_model}.json', 'w') as outfile:
    json.dump(data, outfile, indent=2)

MODEL PATH: gs://gpt-neo-chill/GPT3_2-7B

{'activation_function': 'gelu',
 'ada_epsilon1': '1e-30',
 'ada_epsilon2': 0.001,
 'attention_types': [[['global', 'local'], 16]],
 'attn_dropout': 0,
 'beta1': 0.9,
 'beta2': 0.95,
 'datasets': [['pile', None, None, None]],
 'embed_dropout': 0,
 'eos_id': 50256,
 'epsilon': 1e-08,
 'eval_batch_size': 128,
 'eval_steps': 10,
 'gradient_clipping': 1.0,
 'iterations': 500,
 'layout': 'batch:x,embd:y',
 'lr': 0.00016,
 'lr_decay': 'cosine',
 'lr_decay_end': 300000,
 'mesh_shape': 'x:64,y:4',
 'model_path': 'gs://neo-d/models/GPT3_2-7B',
 'n_ctx': 2048,
 'n_embd': 2560,
 'n_head': 20,
 'n_layer': 32,
 'n_vocab': 50257,
 'opt_name': 'adam',
 'padding_id': 50257,
 'predict_batch_size': 1,
 'predict_steps': 0,
 'recompute_grad': True,
 'res_dropout': 0,
 'scale_by_depth': True,
 'scale_by_in': False,
 'tokens_per_mb_per_replica': 4096,
 'train_batch_size': 512,
 'train_steps': 400000,
 'warmup_steps': 3000,
 'weight_decay': 0}

--->

{'activation_func

Create an example_prompt.txt file and add a prompt. save it in GPTNeo folder

# Model Inference

In [ ]:
!python3 main.py --model $pretrained_model --steps_per_checkpoint 20 --tpu colab --predict --prompt example_prompt.txt

2021-03-28 20:48:56.109851: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Instructions for updating:
non-resource variables are not supported in the long term
Current step 400000
Saving config to gs://gpt-neo-chill/GPT3_2-7B
2021-03-28 20:49:02.056812: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-28 20:49:02.057919: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-28 20:49:02.069338: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-03-28 20:49:02.069433: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (2a32c3b994ec): /proc/driver/nvidia/version does not exist
2021-03-28 20:49:02.496306: I tensorflow/compiler/mlir/mlir_g

# Evaluate Model

In [ ]:
wikitext103_src = "https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip"
!wget $wikitext103_src
!unzip wikitext-103-raw-v1.zip

--2021-03-26 19:12:21--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.77.142
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.77.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191984949 (183M) [application/zip]
Saving to: ‘wikitext-103-raw-v1.zip’

wikitext-103-raw-v1 100%[===================>] 183.09M  68.6MB/s    in 2.7s    

2021-03-26 19:12:24 (68.6 MB/s) - ‘wikitext-103-raw-v1.zip’ saved [191984949/191984949]

Archive:  wikitext-103-raw-v1.zip
   creating: wikitext-103-raw/
  inflating: wikitext-103-raw/wiki.test.raw  
  inflating: wikitext-103-raw/wiki.valid.raw  
  inflating: wikitext-103-raw/wiki.train.raw  


In [ ]:

!mkdir wikitext
!mv /content/GPTNeo/wikitext-103-raw/wiki.test.raw wikitext/wikitext_test.txt

# Tokenize Data
!python data/create_tfrecords.py --input_dir wikitext --name wikitext --files_per 1000 --output_dir wikitext_tokenized --write_dataset_config --processes 1 --wikitext-detokenize

# copy the data to your bucket
if not path_to_cloud_bucket.endswith('/'):
  path_to_cloud_bucket += '/'
copy_loc = path_to_cloud_bucket 
!gsutil -m cp -r wikitext_tokenized $copy_loc
!gsutil ls $path_to_cloud_bucket

2021-03-26 19:12:39.799740: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Writing TFRecord Files to wikitext_tokenized/. Parsed 0 input files. files_written : 0it [00:02, ?it/s]
{'discarded': 0, 'processed': 1, 'successful': 1}
Copying file://wikitext_tokenized/wikitext_0_139.tfrecords [Content-Type=application/octet-stream]...
/ [1/1 files][578.6 KiB/578.6 KiB] 100% Done                                    
Operation completed over 1 objects/578.6 KiB.                                    
gs://gpt-neo-chill/GPT3_2-7B/
gs://gpt-neo-chill/wikitext_tokenized/


In [ ]:
%%writefile configs/dataset_configs/wikitext.json

{
  "path": "",
  "eval_path": "gs://gpt-neo-chill/wikitext_tokenized/*.tfrecords",
  "n_vocab": 50256,
  "tokenizer_is_pretrained": true,
  "tokenizer_path": "gpt2",
  "eos_id": 50256,
  "padding_id": 50257
}

Overwriting configs/dataset_configs/wikitext.json


In [ ]:
# @title Modify config for wikitext. 
  
import json
from pprint import pprint

batch_size = 8 #@param {type:"integer"}
assert pretrained_model is not None
with open(f'configs/{pretrained_model}.json', 'r') as f:
  data = json.load(f)
  pprint(data)
  dset_val = [["wikitext", None, None, None]]
  mods = {
          "datasets": dset_val,
          "eval_steps": 139 // batch_size,
          "train_batch_size": batch_size,
          "eval_batch_size": batch_size,
        }
  data.update(mods)
  print('\n--->\n')
  pprint(data)
  with open(f'configs/{pretrained_model}.json', 'w') as outfile:
    json.dump(data, outfile, indent=2)

{'activation_function': 'gelu',
 'ada_epsilon1': '1e-30',
 'ada_epsilon2': 0.001,
 'attention_types': [[['global', 'local'], 16]],
 'attn_dropout': 0,
 'beta1': 0.9,
 'beta2': 0.95,
 'datasets': [['wikitext', None, None, None]],
 'embed_dropout': 0,
 'eos_id': 50256,
 'epsilon': 1e-08,
 'eval_batch_size': 8,
 'eval_steps': 17,
 'gradient_clipping': 1.0,
 'iterations': 500,
 'layout': 'intermediate_expanded:x,heads:x,memory_length:y,embd:y',
 'lr': 0.00016,
 'lr_decay': 'cosine',
 'lr_decay_end': 300000,
 'mesh_shape': 'x:4,y:2',
 'model_path': 'gs://gpt-neo-chill/GPT3_2-7B',
 'n_ctx': 2048,
 'n_embd': 2560,
 'n_head': 20,
 'n_layer': 32,
 'n_vocab': 50257,
 'opt_name': 'adam',
 'padding_id': 50257,
 'predict_batch_size': 8,
 'predict_steps': 0,
 'recompute_grad': True,
 'res_dropout': 0,
 'scale_by_depth': True,
 'scale_by_in': False,
 'tokens_per_mb_per_replica': 4096,
 'train_batch_size': 8,
 'train_steps': 401000,
 'warmup_steps': 3000,
 'weight_decay': 0}

--->

{'activation_functi

In [ ]:
!python3 main.py --eval --tpu colab --model $pretrained_model

2021-03-26 19:15:28.967534: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Instructions for updating:
non-resource variables are not supported in the long term
Current step 400000
Saving config to gs://gpt-neo-chill/GPT3_2-7B
2021-03-26 19:15:34.992569: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-26 19:15:34.993669: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-26 19:15:35.004895: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-03-26 19:15:35.004958: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (03f26ada248b): /proc/driver/nvidia/version does not exist
2021-03-26 19:15:35.295161: I tensorflow/compiler/mlir/mlir_g